## This notebook is intended for imputation of missing values in time series datasets obtained from wave buoys using attention based combination of Long-Short Term Memory (LSTM) and Convolutional Neural networks (CNNs).

### Data preparation

The data is obtained from the [CDIP buoys](https://cdip.ucsd.edu/m/stn_table/) website, where wave surface elevation data from different surface buoys in the form of netcdf files are obtained by using a custom $\text{MATLAB}$ script. The netcdf files contain historic as well as real-time data of different measured wave variables. 

- **The objective was to predict possible sudden high-amplitude waves (wave annomalies (better known as *rogue waves*) that might be present in the missing piece of data. These waves occur suddenly in the time scale of seconds, hence it made sense to utilize the records of an observed variable which has a high sampling rate.**
- Keeping this in mind, the wave surface elevation data which has a sampling rate of 1.28*Hz* was used. Other buoy variables, either did not directly capture the wave magnitudes or had a much lower sampling rate (order of minutes).
- Thus, it made sense to use surface wave elevation data for the wave data imputation exercise.
- The data is acquired in the form of windows spanning from 20 to 25 minutes.

 ### Data pre-processing
- The wave peaks and troughs are filtered, since they embody the extreme values in the time series.
- These are then broken down to two sections with a specific duration of data missing between them. The objective is to impute *these peakls in the time series data*.   
- The $\text{MATLAB}$ script above employs a wave modelling equation to fit the obtained data peak values in the time windows. This involves the use of a *regularization parameter $\alpha$* which penalizes the extreme values of wave magnitudes.
- Along with this, number of Fourier components used for breaking down the wave data were also investigated to give the best fit.
- The fit data was then broken down into individual time series datasets corresponding to the number of Fourier components. These qualify as the final processed datasets to be used for the ML and other quantitative modelling techniques.
- To summarize, we had ***100 data windows*** and ***N = 33 components***. This resulted in **33 datasets corresponding to the portion preceding the missing data, 33 datasets corresponding to the portion following the missing data for each window**.
- Finally, a lagged table was created for each of these components and **33 total datasets for 1 window** were created by combining the trailing and leading portions. A lag value of ***M=200*** was chosen after parametric studies. This corresponded to the case which resulted in the **lowest validation error** for a sample window.
- Tools used: ***pandas***, ***Numpy***      

### Attention-based CNN+LSTM model architecture
- The model architecture consists of three parts.
- The initial time series dataset is fed to a **LSTM model**.
- In parallel, a scalogram capturing the time-frequency relationship in the time series is also obtained through wavelet analysis.
- This **scalogram is fed to a CNN model** to extract the different features inherent in the time-frequency relationship.
- The **LSTM is responsible for capturing the long-term trends in the time series**. The **CNN**, on the other hand, **is reponsible for capturing the localized features**, through proper selection of the kernel sizes and strides in the CNN. 3 layers of convolution features have been used.
- In the second part of the model, the output from the last layer of the CNN model is combined with the output from the last hidden layer of the LSTM model. The **combined features are then fed to an attention mechanism using a Multi-Layer Perceptron (MLP) network**.
- In the third part, the **output from this attention process is combined with the output from the last hidden layer of the LSTM and fed to a MLP for the final output at the next time step**. This process is continued iteratively till the intended imputation duration. 

![cnn_lstm](cnn_lstm.jpg)
**Architecture of the attentive CNN + LSTM network used for the imputation of missing entries in the present studies. The input sequence, on one hand is fed to a LSTM. Along with this, the time series sequence is used to create a scalogram, which is fed to a CNN model. The outputs from these models are utilized to get attention scores of the CNN features, which are then fed alongside the outputs of the last LSTM hidden layer to an MLP model to get the final prediction.**

### CNN+LSTM Model Workflow 
- The model workflow comprises of the following processes:
    1. Reading the pre-processed data using **pandas** and **Numpy**.
    3. Generating the dataset using a lagged table to be used as input for the LSTM model. 
    4. Using wavelet analysis to generate scalogram plots to be used as inputs for the CNN model. 
    5. Standardizing the features and creating the train, validation and test datasets.
    6. Training the above model architecture using custom optimizer class on the data using **PyTorch**.
    7. Validating the trained models through a early stopping mechanism
    8. Storing the trained model.
    9. Evaluating the test data on the trained model.
- The workflow thus discussed constitutes **the imputed values for 1 of the 33 components**. A for-loop is used for imputation of missing values for all the 33 components.

#### Generating the datasets for feeding into the LSTM and the CNN+LSTM models 

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn

import torch.nn as nn
import matplotlib.pyplot as plt
import math
import torchvision
import torchvision.transforms as transforms
from PIL import Image
transform = transforms.ToTensor()
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import datetime as datetime
from timeit import default_timer as timer
import pywt
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

transform = transforms.ToTensor()
transform2 = transforms.Resize((54,55))


# In[3]:


for i in range(1,3):
    data_pre = pd.read_csv(f"Slow_amp_pre_{i}.csv", header=None)
    data_post = pd.read_csv(f"Slow_amp_post_{i}.csv", header=None)
    data_whole = pd.read_csv(f"Slow_amp_whole_{i}.csv", header=None)
    
    n_rows = data_pre.shape[0]
    n_cols = data_whole.shape[1] - (data_pre.shape[1] + data_post.shape[1])

    data_miss = pd.DataFrame(np.zeros([n_rows, n_cols])*np.nan)
    
    data_pre_vals = data_pre[:].values
    data_post_vals = data_post[:].values
    data_whole_vals = data_whole[:].values
    
    data_test = scaler.fit_transform(data_whole_vals.reshape(-1,1)).reshape(data_whole_vals.shape[0],data_whole_vals.shape[1])
    missing_len = data_miss.shape[1]
    
    #Predictions_pre = np.zeros([data_miss.shape[0], data_miss.shape[1]])
    #Values = np.zeros([data_miss.shape[0], data_miss.shape[1]])

    dummy_data = pd.concat([data_pre, data_post], axis=1,ignore_index=True)
    data = scaler.fit_transform(dummy_data[:].values.reshape(-1,1)).reshape(dummy_data.shape[0],dummy_data.shape[1])
    pre_data_scaled = data[:,:data_pre.shape[1]]
    pre_shape = pre_data_scaled.shape
    #data = data[:3,-254:]
    
    input_len = 200
    output_len = 1  
    input_dim = input_len
    output_dim = output_len
    tuple_shape = (53, 54)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    L1 = data.shape[1] - input_len
    train_len = int(0.7 * L1)
    val_len = L1 - train_len
    
    X_train_pre_lstm_tensor = torch.zeros(len(data), train_len, input_len)
    Y_train_pre_tensor = torch.zeros(len(data), train_len, output_len)
    X_train_pre_CNN_tensor = torch.zeros(len(data), train_len, 3 , 53, 54)
    
    
    X_val_pre_lstm_tensor = torch.zeros(len(data), val_len, input_len)
    Y_val_pre_tensor = torch.zeros(len(data), val_len, output_len)
    X_val_pre_CNN_tensor = torch.zeros(len(data), val_len, 3 , 53, 54)

    for j in range(len(data)):
        X_pre = np.zeros([L1, input_len])
        Y_pre = np.zeros([L1, output_len])

        for k in range(L1):
            X_pre[k,:] = data[j,k:k+input_len]
            Y_pre[k,:] = data[j,k+input_len:k+input_len+output_len]

        Train_X_pre = X_pre[:train_len]
        Train_Y_pre = Y_pre[:train_len]
        
        Val_X_pre = X_pre[train_len:]        
        Val_Y_pre = Y_pre[train_len:]
    
        X_train_pre_lstm_tensor[j] = torch.Tensor(Train_X_pre.copy())
        Y_train_pre_tensor[j] = torch.Tensor(Train_Y_pre.copy())
    
        X_val_pre_lstm_tensor[j] = torch.Tensor(Val_X_pre.copy())
        Y_val_pre_tensor[j] = torch.Tensor(Val_Y_pre.copy())
        
        for m in range(train_len):            
            time = (1/1.28)*np.arange(0, len(Train_X_pre[m]))
            signal = Train_X_pre[m]
            scales = np.arange(1, 256)
            dt = time[1] - time[0]
            waveletname = 'cmor' 
            cmap = plt.cm.jet
            [coefficients, frequencies] = pywt.cwt(signal, scales, waveletname, dt)
            power = (abs(coefficients)) ** 2    
            p_h = 0.05
            p_l = 0.0
            levs = np.arange(p_l,p_h,(p_h - p_l)/100)    
            fig = plt.figure(figsize=(0.7,0.7))
            im = plt.contourf(time, frequencies, power, cmap = cmap, vmin=p_l, vmax=p_h, levels=levs, extend='both')
            plt.axis('off')
            plt.xlim([20, 150])
            plt.ylim([0.005, 0.04])
            plt.savefig(f'Sample{i}.jpeg', bbox_inches='tight',pad_inches = 0)
            plt.close()
            #print(f'm:{m}')
        
            image = Image.open(f'Sample{i}.jpeg')
            tensor = transform(image)
            X_train_pre_CNN_tensor[j,m] = tensor

        for m in range(val_len) :  
            time = (1/1.28)*np.arange(0, len(Val_X_pre[m]))
            signal = Val_X_pre[m]
            scales = np.arange(1, 256)
            dt = time[1] - time[0]
            waveletname = 'cmor' 
            cmap = plt.cm.jet
            [coefficients, frequencies] = pywt.cwt(signal, scales, waveletname, dt)
            power = (abs(coefficients)) ** 2   
            p_h = 0.05
            p_l = 0.0
            levs = np.arange(p_l,p_h,(p_h - p_l)/100)    
            fig = plt.figure(figsize=(0.7,0.7))
            im = plt.contourf(time, frequencies, power, cmap = cmap, vmin=p_l, vmax=p_h, levels=levs, extend='both')
            plt.axis('off')
            plt.xlim([20, 150])
            plt.ylim([0.005, 0.04])
            plt.savefig(f'SampleVal{i}.jpeg', bbox_inches='tight',pad_inches = 0)
            plt.close()
        
            image = Image.open(f'SampleVal{i}.jpeg')
            tensor = transform(image)
            X_val_pre_CNN_tensor[j,m] = tensor
            
        #print(f'j:{j}')
            
    torch.save(X_train_pre_lstm_tensor, f'X_Train_LSTM_Tensor_{i}.pt')
    torch.save(Y_train_pre_tensor, f'Y_Train_Tensor_{i}.pt')
    torch.save(X_train_pre_CNN_tensor, f'X_CNN_LSTM_Train_Tensor_{i}.pt')
    
    torch.save(X_val_pre_lstm_tensor, f'X_Val_LSTM_Tensor_{i}.pt')
    torch.save(Y_val_pre_tensor, f'Y_Val_Tensor_{i}.pt')
    torch.save(X_val_pre_CNN_tensor, f'X_CNN_LSTM_Val_Tensor_{i}.pt')

#### Importing the required modules

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn

import torch.nn as nn
import matplotlib.pyplot as plt
import math

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import datetime as datetime
from timeit import default_timer as timer
import pywt
from sklearn.preprocessing import MinMaxScaler
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import LogSoftmax
from torch import flatten

#### Transforms for creating images to PyTorch tensors 

In [ ]:
import torchvision
import torchvision.transforms as transforms
from PIL import Image
transform = transforms.ToTensor()
transform2 = transforms.Resize((53,54))

#### CNN+LSTM model class encompassing the LSTM, CNN+LSTM and the attention mechanism

In [ ]:
class CNN_LSTM_Module(nn.Module):
    def __init__(self, oc1, s_conv, ks, dil1, dil2, ts, i_dim, h_dim, l_dim, d_prob, mlp_hdim1, mlp_odim, a_net_feature):
        self.oc1 = oc1
        self.s_conv = s_conv
        self.ks = ks
        self.ts = ts
        self.dil1 = dil1
        self.dil2 = dil2
        
        self.i_dim = i_dim
        self.h_dim = h_dim
        self.l_dim = l_dim
        self.d_prob = d_prob
        self.mlp_hdim1 = mlp_hdim1
        self.mlp_odim = mlp_odim
        self.a_net_feature = a_net_feature

        super(CNN_LSTM_Module, self).__init__()

        '''Convolution part of the model'''
        
        self.convD1_1 = Conv2d(in_channels=3, out_channels=oc1, dilation = dil1, kernel_size=ks)
        self.convD1_2 = Conv2d(in_channels=oc1, out_channels=oc1, dilation = dil1, kernel_size=ks)
        
        self.convD2_1 = Conv2d(in_channels=3, out_channels=oc1, dilation = dil2, kernel_size=ks)
        self.convD2_2 = Conv2d(in_channels=oc1, out_channels=oc1, dilation = dil2, kernel_size=ks)
        
        self.relu = ReLU()
        
        ### DILATION VAL 1 ###
        
        ### initialize first set of CONV => RELU => layers ###
        size1_1 = math.floor((ts[0] - dil1 * (ks-1) - 1)/s_conv)+1
        size1_2 = math.floor((ts[1] - dil1 * (ks-1) - 1)/s_conv)+1
        
        ### initialize second set of CONV => RELU => layers ###
        size1_1 = math.floor((size1_1 - dil1 * (ks-1) - 1)/s_conv)+1
        size1_2 = math.floor((size1_2 - dil1 * (ks-1) - 1)/s_conv)+1
        
        ### initialize third set of CONV => RELU => layers ###
        size1_1 = math.floor((size1_1 - dil1 * (ks-1) - 1)/s_conv)+1
        size1_2 = math.floor((size1_2 - dil1 * (ks-1) - 1)/s_conv)+1
        
        """LSTM part of the model"""
        # LSTM layers
        self.lstm = nn.LSTM(i_dim, h_dim, l_dim, batch_first=True, dropout=0.4)

        """Attention module of the network"""
        out_features_Layer1 = a_net_feature
        
        D1_in_features_Layer1 = (size1_1 * size1_2) + (h_dim * l_dim)
        self.attentionD1 = Linear(D1_in_features_Layer1, out_features_Layer1)

        self.tanh = nn.Tanh()

        in_features_Layer2 = self.a_net_feature
        out_features_Layer2 = 1
        self.attention2 = Linear(in_features_Layer2, out_features_Layer2)

        self.smax = nn.Softmax(dim=1)

        """Fusion and Predictions using Multi Layer Perceptron"""
        
        #fusion_input_dim = (size1_1 * size1_2) + (size2_1 * size2_2) + (self.h_dim * self.l_dim)
        fusion_input_dim = (size1_1 * size1_2) + (self.h_dim * self.l_dim)
        fusion_hidden_dim1 = math.floor(fusion_input_dim/2)
        fusion_output_dim = self.mlp_odim

        self.fc1 = nn.Linear(fusion_input_dim, fusion_hidden_dim1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(self.d_prob)
        self.relu = ReLU()
        self.fc2 = nn.Linear(fusion_hidden_dim1, fusion_output_dim)

    def forward(self, x1, x2):
        '''CNN 2D convolution using scaleograms'''
        
        #### 1st dilation variant ####
        
        xD1 = self.convD1_1(x1)
        xD1 = self.relu(xD1)

        xD1 = self.convD1_2(xD1)
        xD1 = self.relu(xD1)

        xD1 = self.convD1_2(xD1)
        xD1 = self.relu(xD1)

        xD1_inter = xD1.reshape(x2.size(0), -1, xD1.shape[2]*xD1.shape[3])

        '''Hidden state prediction using LSTM for the global trend'''
        h0 = torch.zeros(self.l_dim, x2.size(0), self.h_dim).requires_grad_().to(device)

        # Initializing cell state for first input with zeros
        c0 = torch.zeros(self.l_dim, x2.size(0), self.h_dim).requires_grad_().to(device)

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        # Forward propagation by passing in the input, hidden state, and cell state into the model
        out, (hn, cn) = self.lstm(x2, (h0.detach(), c0.detach()))
        
        #print("The size of the hidden LSTM output directly has the shape of {}".format(hn.size()))
        
        #a = torch.Tensor([[]]).to(device)
        #for i in range(hn.shape[0]):
        #    a = torch.cat((a,hn[i]),1)
        
        hn = torch.cat((hn[0], hn[1]), 1)
        
        #hn = hn.reshape([x2.size(0),-1])
        #print("The size of the hidden LSTM output has the shape of {}".format(hn.size()))
        
        hn_rec1 = hn.reshape(hn.shape[0],1,hn.shape[1])
        for l in range(1, xD1_inter.shape[1]):
            hn_rec1 = torch.cat((hn_rec1, hn.reshape(hn.shape[0],1,hn.shape[1])),1)
        
        #hn_rec2 = hn.reshape(hn.shape[0],1,hn.shape[1])
        #for l in range(1, xD2_inter.shape[1]):
        #    hn_rec2 = torch.cat((hn_rec2, hn.reshape(hn.shape[0],1,hn.shape[1])),1)

        #print("The size of the hidden LSTM output after concatenation has the shape of {}".format(hn_rec.size()))

        #hn_rec = hn_rec.reshape([-1,x1_inter.size(1),hn.size(1)])
        #print("The size of the reconstructed LSTM hidden layer has the shape of {}".format(hn_rec.size()))
        
        '''Attention module using the hidden states and the CNN module'''
        xd1 = torch.concat((xD1_inter,hn_rec1),dim=2)
        #xd2 = torch.concat((xD2_inter,hn_rec2),dim=2)
        
        #print("The size of the augmented input attention layer has the shape of {}".format(x.size()))
        
        xdil1 = self.attentionD1(xd1)
        #xdil2 = self.attentionD2(xd2)
        
        #print("The size of the input after the first attention layer has the shape of {}".format(x.size()))
        
        xdil1 = self.tanh(xdil1)
        #xdil2 = self.tanh(xdil2)
        
        xdil1 = self.attention2(xdil1)
        #xdil2 = self.attention2(xdil2)
        
        #print("The size of the input after the second attention layer has the shape of {}".format(x.size()))
        
        xdil1 = self.smax(xdil1)
        #xdil2 = self.smax(xdil2)
        
        #print("The size of the input after the softmax layer has the shape of {}".format(x.size()))
        
        #print(torch.transpose(x1_inter,1,2).shape)
        
        xdil1 = torch.bmm(torch.transpose(xD1_inter,1,2),xdil1).to(device)
        #xdil2 = torch.bmm(torch.transpose(xD2_inter,1,2),xdil2).to(device)
        
        #print("The size of the input after the weighted sum has the shape of {}".format(x.size()))
        
        xdil1 = xdil1.reshape([x2.size(0),-1])
        #xdil2 = xdil2.reshape([x2.size(0),-1])
        
        #print("The size of the reshaped input layer before MLP has the shape of {}".format(x.size()))

        '''Fusion and prediction using MLP'''
        #x_MLP = torch.concat((xdil1, xdil2, hn),dim=1)
        x_MLP = torch.concat((xdil1, hn),dim=1)
        
        #print("The size of the augmented input MLP layer` has the shape of {}".format(x_MLP.size()))
        
        x_MLP = self.fc1(x_MLP)
        
        #print("The size of the input after the first hidden MLP layer has the shape of {}".format(x_MLP.size()))
        
        x_MLP = self.relu(x_MLP)
        
        x_MLP = self.dropout(x_MLP)
        
        x_MLP = self.fc2(x_MLP)
        
        output = x_MLP
        
        #print("The size of the output after the MLP layer has the shape of {}".format(output.size()))
        
        return output

#### Custom optimizer class to include the training, validation and loss plotting methods

In [ ]:
class Optimization:
    """Optimization is a helper class that allows training, validation, prediction.
    """
    def __init__(self, model, loss_fn, optimizer, patience, min_delta = 1e-5):

        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []
        self.counter = 0
        self.min_delta = min_delta
        self.min_validation_loss = np.inf
        self.patience = patience
        
    def train_step(self,x1,x2,y):
        self.model.train()

        yhat = self.model(x1,x2)
        loss = self.loss_fn(y, yhat)

        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()

        return loss.item()
    
    def earlyStop(self, validation_loss):
        if validation_loss < (self.min_validation_loss - self.min_delta):
            self.min_validation_loss = validation_loss
            self.counter = 0
            
        elif validation_loss >= (self.min_validation_loss - self.min_delta):
            self.counter +=1
            if self.counter >= self.patience:
                return True
            return False

    def train(self, train_loader, val_loader, batch_size, n_epochs, mode, n_features, output_dim):

        model_path = f'cnn2d+lstm.pt'
        break_out_flag = False

        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            for x1_batch, x2_batch, y_batch in train_loader:
                x2_batch = x2_batch.view([batch_size, -1, n_features]).to(device)
                x1_batch = x1_batch.to(device)
                y_batch = y_batch.to(device)
                yhat = self.model(x1_batch, x2_batch)
                loss = self.train_step(x1_batch, x2_batch, y_batch)
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x1_val, x2_val, y_val in val_loader:
                    x2_val = x2_val.view([batch_size, -1, n_features]).to(device)
                    x1_val = x1_val.to(device)
                    y_val = y_val.to(device)
                    self.model.eval()
                    yhat = self.model(x1_val, x2_val)
                    val_loss = self.loss_fn(y_val, yhat).item()
                    batch_val_losses.append(val_loss)
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)
                if self.earlyStop(validation_loss):
                    break_out_flag = True
                    break               
            
            if break_out_flag:
                torch.save(self.model.state_dict(), model_path)
                break

            if (epoch <= 10) | (epoch % 25 == 0):
                print(
                    f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}"
                )
        #torch.save(self.model.state_dict(), model_path)

        
    def evaluate(self, x, test, training_len, output_len, missing_len, sample):
        with torch.no_grad():
            predictions = []
            values = []
            for j in range(len(test)):
                val = test[j].to(device).cpu()
                values.append(val.detach().numpy())
            
            for i in range(missing_len):
                x = x.to(device)
                self.model.eval()
                #x_test = x.view([1, -1, training_len]).to(device)
                x_lstm_test = x.view([1, -1, training_len]).to(device)

                x_a = x.cpu()
                x_arr = np.array(x_a)
                x_arr = x_arr.reshape(len(x_arr))
                #print(x_arr.shape)

                time =  time = (1/1.28)*np.arange(0, len(x_arr))
                signal = x_arr
                scales = np.arange(1, 256)

                dt = time[1] - time[0]
                waveletname = 'cmor'
                cmap = plt.cm.jet
                [coefficients, frequencies] = pywt.cwt(signal, scales, waveletname, dt)
                power = (abs(coefficients)) ** 2

                p_h = 0.05
                p_l = 0.0
                levs = np.arange(p_l,p_h,(p_h - p_l)/100)    
                fig = plt.figure(figsize=(0.7,0.7))
                im = plt.contourf(time, frequencies, power, cmap = cmap, vmin=p_l, vmax=p_h, levels=levs, extend='both')
                plt.axis('off')
                plt.xlim([20, 150])
                plt.ylim([0.005, 0.04])
                plt.savefig(f'SamplePred{sample}.jpeg', bbox_inches='tight',pad_inches = 0)
                plt.close()

                image = Image.open(f'SamplePred{sample}.jpeg')

                resized_image = transform2(image)
                tensor = transform(resized_image)
                
                tensor = tensor.unsqueeze(0).to(device)
                #print(tensor.size())

                yhat = self.model(tensor, x_lstm_test)
                yint = torch.reshape(yhat,(output_len,1))
                
                y_int = yint.to(device).cpu()
                
                predictions.append(y_int[-1].detach().numpy())
                x=torch.reshape(x,(training_len,1))
                x = torch.cat((x,yint[-1].reshape(1,1)),0)
                x = x[-training_len:]
            
        preds =  torch.reshape(torch.Tensor(predictions),(-1,1))
        
        return np.asarray(values), np.asarray(preds) 
    
    def evaluate2(self, x, test, training_len, missing_len):
        with torch.no_grad():
            predictions = []
            values = []
            for j in range(len(test)):
                val = test[j].to(device).cpu()
                values.append(val.detach().numpy())
            
            num = len(test) % missing_len
            print(num)
            if (num == 0):    
                for i in range(math.floor(len(test)/missing_len)):
                    x = x.to(device)
                    self.model.eval()
                    #x_test = x.view([1, -1, training_len]).to(device)
                    x_lstm_test = x.view([1, -1, training_len]).to(device)

                    x_a = x.cpu()
                    x_arr = np.array(x_a)
                    x_arr = x_arr.reshape(len(x_arr))
                    #print(x_arr.shape)

                    time =  time = (1/48)*np.arange(0, len(x_arr))
                    signal = x_arr
                    scales = np.arange(1, 512)

                    dt = time[1] - time[0]
                    waveletname = 'cmor'
                    cmap = plt.cm.jet
                    [coefficients, frequencies] = pywt.cwt(signal, scales, waveletname, dt)
                    power = (abs(coefficients)) ** 2

                    p_h = 0.1
                    p_l = 0.0

                    levs = np.arange(p_l,p_h,(p_h - p_l)/100)

                    fig = plt.figure(figsize=(1,1))
                    im = plt.contourf(time, frequencies, power, cmap = cmap, vmin=p_l, vmax=p_h, levels=levs, extend='both')
                    plt.axis('off')
                    plt.savefig(f'Sample2.jpeg', bbox_inches='tight',pad_inches = 0)
                    plt.close()

                    image = Image.open(f'Sample2.jpeg')

                    resized_image = transform2(image)
                    tensor = transform(resized_image)
                
                    tensor = tensor.unsqueeze(0).to(device)
                    #print(tensor.size())

                    yhat = self.model(tensor, x_lstm_test)
                    yint = torch.reshape(yhat,(missing_len,1))
                
                    y_int = yint.to(device).cpu()
                
                    predictions.append(y_int.detach().numpy())
                    x=torch.reshape(x,(training_len,1))
                    x = torch.cat((x,yint),0)
                    x = x[-training_len:]
                    
            else:
                for i in range(math.floor(len(test)/missing_len)+1):
                    x = x.to(device)
                    self.model.eval()
                    #x_test = x.view([1, -1, training_len]).to(device)
                    x_lstm_test = x.view([1, -1, training_len]).to(device)

                    x_a = x.cpu()
                    x_arr = np.array(x_a)
                    x_arr = x_arr.reshape(len(x_arr))
                    #print(x_arr.shape)

                    time =  time = (1/48)*np.arange(0, len(x_arr))
                    signal = x_arr
                    scales = np.arange(1, 512)

                    dt = time[1] - time[0]
                    waveletname = 'cmor'
                    cmap = plt.cm.jet
                    [coefficients, frequencies] = pywt.cwt(signal, scales, waveletname, dt)
                    power = (abs(coefficients)) ** 2

                    p_h = 0.1
                    p_l = 0.0

                    levs = np.arange(p_l,p_h,(p_h - p_l)/100)

                    fig = plt.figure(figsize=(1,1))
                    im = plt.contourf(time, frequencies, power, cmap = cmap, vmin=p_l, vmax=p_h, levels=levs, extend='both')
                    plt.axis('off')
                    plt.savefig(f'Sample2.jpeg', bbox_inches='tight',pad_inches = 0)
                    plt.close()

                    image = Image.open(f'Sample2.jpeg')

                    resized_image = transform2(image)
                    tensor = transform(resized_image)
                
                    tensor = tensor.unsqueeze(0).to(device)
                    #print(tensor.size())

                    yhat = self.model(tensor, x_lstm_test)
                    yint = torch.reshape(yhat,(missing_len,1))
                
                    y_int = yint.to(device).cpu()
                
                    predictions.append(y_int.detach().numpy())
                    x=torch.reshape(x,(training_len,1))
                    x = torch.cat((x,yint),0)
                    x = x[-training_len:]
                    
        preds =  torch.reshape(torch.Tensor(predictions),(-1,1))
        return np.asarray(values), np.asarray(preds)


    def plot_losses(self, output_len):
        """The method plots the calculated loss values for training and validation
        """
        #np.savetxt(f"CNN_LSTM_dilatedConcatenated_79_mean_train.out", self.train_losses, fmt='%1.4e')
        #np.savetxt(f"CNN_LSTM_dilatedConcatenated_79_mean_val.out", self.val_losses, fmt='%1.4e')
        
        #plt.figure(figsize=[10,8])
        #plt.plot(self.train_losses, label="Training loss")
        #plt.plot(self.val_losses, label="Validation loss")
        #plt.legend()
        #plt.title(f"Losses for dilated concatenation using mean reduction for 79 steps")
        #plt.grid()
        #plt.show()
        #plt.savefig(f'CNN_LSTM Losses comparisons for dilatedConcatenated_79_mean over epochs.png',dpi=300)
        #plt.close()

#### Creating the custom datasets

In [ ]:
from torch.utils.data import Dataset

class CNN_LSTM_Dataset(Dataset):

    def __init__(self, images, entries, labels):
        self.X1 = images
        self.X2 = entries
        self.Y = labels

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, idx):
        data1 = self.X1[idx,:]
        data2 = self.X2[idx,:]

        data_labels = self.Y[idx,:]

        return (data1, data2, data_labels)

#### Working on the main data
- The data is read from the created datasets here. 
- The diffent functions are called here. 
- The data is broken down into training, validation datasets after scaling them. 
- The missing data is taken as the test dataset. 
- The hyperparameters shown here are arrived at after extensive hyperparameter tuning *(convolutional features, hidden dimension, layer dimension, mlp hidden dimenison)*
- Finally, the missing portion was imputed through iterative single step ahead predictions and the predictions and values stored for comparisons.

In [ ]:
input_len = 200
output_len = 1
    
input_dim = input_len
output_dim = output_len

ks = 3
hidden_dim = 200
layer_dim = 2
dropout_prob = 0.3
a_net_feature = 32
mlp_hiddendim1 = 100
mlp_odim = output_len

oc1 = 150

dilation1 = 1
dilation2 = 7

tuple_shape = (53, 54)
weight_decay = 1e-3
    
for i in range(1,3):
    data_pre = pd.read_csv(f"Slow_amp_pre_{i}.csv", header=None)
    data_post = pd.read_csv(f"Slow_amp_post_{i}.csv", header=None)
    data_whole = pd.read_csv(f"Slow_amp_whole_{i}.csv", header=None)
    
    n_rows = data_pre.shape[0]
    n_cols = data_whole.shape[1] - (data_pre.shape[1] + data_post.shape[1])

    data_miss = pd.DataFrame(np.zeros([n_rows, n_cols])*np.nan)
    
    data_pre_vals = data_pre[:].values
    data_post_vals = data_post[:].values
    data_whole_vals = data_whole[:].values
    
    data_test = scaler.fit_transform(data_whole_vals.reshape(-1,1)).reshape(data_whole_vals.shape[0],data_whole_vals.shape[1])
    missing_len = data_miss.shape[1]
    
    Predictions_pre = np.zeros([data_miss.shape[0], data_miss.shape[1]])
    Values = np.zeros([data_miss.shape[0], data_miss.shape[1]])

    dummy_data = pd.concat([data_pre, data_post], axis=1,ignore_index=True)
    data = scaler.fit_transform(dummy_data[:].values.reshape(-1,1)).reshape(dummy_data.shape[0],dummy_data.shape[1])
    pre_data_scaled = data[:,:data_pre.shape[1]]
    pre_shape = pre_data_scaled.shape
    #data = data[:3,-254:]
    
    input_len = 200
    output_len = 1  
    input_dim = input_len
    output_dim = output_len
    tuple_shape = (53, 54)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    x_lstm_train = torch.load( f'X_Train_LSTM_Tensor_{i}.pt')
    y_train = torch.load(f'Y_Train_Tensor_{i}.pt')
    x_cnn_train = torch.load( f'X_CNN_LSTM_Train_Tensor_{i}.pt')
    
    x_lstm_val = torch.load( f'X_Val_LSTM_Tensor_{i}.pt')
    y_val = torch.load(f'Y_Val_Tensor_{i}.pt')
    x_cnn_val = torch.load( f'X_CNN_LSTM_Val_Tensor_{i}.pt')
    
    for j in range(len(data)):
        torch.manual_seed(2)
    
        train_pre_eta = CNN_LSTM_Dataset(x_cnn_train[j], x_lstm_train[j], y_train[j])
        val_pre_eta = CNN_LSTM_Dataset(x_cnn_val[j], x_lstm_val[j], y_val[j])
    
        start = timer()
    
        model = CNN_LSTM_Module(oc1 = oc1, s_conv = 1, ks = ks, dil1 = dilation1, dil2 = dilation2, ts = tuple_shape, i_dim = input_dim, h_dim = hidden_dim, l_dim = layer_dim, d_prob = dropout_prob, mlp_hdim1 = mlp_hiddendim1, mlp_odim = output_len, a_net_feature = a_net_feature)
        model = model.to(device)

        batch_size = 32
        n_epochs = 500

        learning_rate = 1e-5
        loss_fn = nn.MSELoss(reduction="mean")
        #loss_fn = MeanCubeLoss()

        optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
        Ftrain_loader = DataLoader(train_pre_eta, batch_size=batch_size, shuffle=False, drop_last=True)
        Fval_loader = DataLoader(val_pre_eta, batch_size=batch_size, shuffle=False, drop_last=True)
    
        opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer, patience = 30)
        opt.train(Ftrain_loader, Fval_loader, batch_size=batch_size, n_epochs=n_epochs, mode=i, n_features=input_dim, output_dim = output_dim)
        opt.plot_losses(output_dim)
            
        end = timer()

        dur = (end-start)/60
        print(f'The total duration for the training is {dur} minutes')

        X_Test = np.asarray(data_test[j,data_pre_vals.shape[1]-input_len:data_pre_vals.shape[1]])
        Y_Test = np.asarray(data_test[j,data_pre_vals.shape[1]:data_pre_vals.shape[1]+missing_len])

        Test_features = torch.Tensor(X_Test)
        Test_targets = torch.Tensor(Y_Test)

        #model = CNN_LSTM_Module(oc1 = oc1, s_conv = 1, ks = ks, dil1 = dilation1, dil2 = dilation2, ts = tuple_shape, i_dim = input_dim, h_dim = hidden_dim, l_dim = layer_dim, d_prob = dropout_prob, mlp_hdim1 = mlp_hiddendim1, mlp_odim = output_len, a_net_feature = a_net_feature)
        #model = model.to(device)

        #PATH = f'cnn2d+lstm.pt'
        #model.load_state_dict(torch.load(PATH))
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
        bl1 = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer, patience = 50)
        values, preds = bl1.evaluate(Test_features,Test_targets, input_len, output_len, missing_len,i)
    
        num = len(Test_targets) % output_len

        if (num != 0):
            preds = preds[:len(Test_targets)]
 
        p = np.asarray(preds).reshape(missing_len)
        Predictions_pre[j,:] = p
        Values[j,:] = values
    
    Preds_rescaled = scaler.inverse_transform(Predictions_pre.reshape(-1,1)).reshape(Predictions_pre.shape[0],Predictions_pre.shape[1])
    Vals_rescaled = scaler.inverse_transform(Values.reshape(-1,1)).reshape(Predictions_pre.shape[0],Predictions_pre.shape[1])
    
    np.savetxt(f"Preds_pre_cnn+lstm_{i}.out", Preds_rescaled)
    np.savetxt(f"Vals_{i}.out", Vals_rescaled)

#### The ML workflow shown here depicts the process for 2 of the windows. The same process was followed for the other windows.
- Following this, the wave peaks and troughs in the missing portion were reconstructed.
- The imputed values for the 33 components were used to give the surface elevation value through a reconstruction process utilizing the same wave modeling equation.
- Finally, the predicted and the true peak values were compared for the entire window and a mean absolute error was used to give an estimate of the imputation efficiency.

##### The comparisons of the CNN+LSTM model with other imputation approaches are shown in a different notebook.